<a href="https://colab.research.google.com/github/MustafizSaadi/DMLab/blob/main/DataMiningAssignment1Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [ ]:
from sortedcontainers import SortedList, SortedSet, SortedDict 
import re

In [ ]:
class Node:
  def __init__(self, parent, val):
    self.parent = parent
    self.val = val
    self.child_dict = SortedDict()
    self.child = []
    self.sup = 0
    self.candidate = True
 
  def get_parent(self):
    return self.parent
 
  def get_val(self):
    return self.val
 
  def get_sup(self):
    return self.sup
 
  def set_sup(self):
    self.sup = -1
 
  def inc_sup(self):
    self.sup += 1
 
  def is_candidate(self):
    return self.candidate
 
  def negate_candidate(self):
    self.candidate = False
 
  def insert_child(self, node):
    cnt = len(self.child_dict)
    self.child_dict[node.get_val()] = cnt + 1
    self.child.append(node)
    return cnt + 1
 
  def delete_child(self, node):
    # print(1)
    idx = self.child_dict[node.get_val()]
    del self.child_dict[node.get_val()]
    for c in self.child_dict:
      if c > node.get_val():
        self.child_dict[c] -= 1
    self.child.pop(idx-1)

In [ ]:
class trie:
  def __init__(self, root, answer, candidate_itemset):
    self.root = root
    self.answer = answer
    self.candidate_itemset = candidate_itemset
 
    for itemset in candidate_itemset:
      cur_node = root
      for item in itemset:
        if item in cur_node.child_dict:
          idx = cur_node.child_dict[item]
          cur_node = cur_node.child[idx-1]
        else:
          child_node = Node(cur_node, item)
          idx = cur_node.insert_child(child_node)
          cur_node = cur_node.child[idx-1]
 
 
  def insert(self, parent, node):
    parent.insert_child(node)
 
  def delete(self, node):
    parent = node.get_parent()
    # print(parent.get_val())
    if not parent == None:
      parent.delete_child(node)
 
  def get_answer(self):
    return self.answer

In [ ]:
def print_path(node, string):
  if len(node.child) == 0 :
    print(string)
    return
  for child in node.child:
    print_path(child, string + child.get_val())

In [ ]:
def update_tree(node, string, min_sup, itemset_2, bucket):
  if len(node.child) == 0 :
    if node.get_sup() >= min_sup :
      itemset_2.add(string)
    else:
      node.set_sup()
      # print(node.get_parent().get_val())
      bucket.append(node)
    return
  flag = False
  for child in node.child:
    update_tree(child, string + child.get_val(), min_sup, itemset_2, bucket)
    if not child.get_sup() == -1:
      flag = True
  if not flag:
    node.set_sup()
    bucket.append(node)


In [ ]:
def generate_candidate_2(itemset_1):
  cnt = len(itemset_1)
  print(cnt)
  answer = SortedSet()
  for i in range(cnt):
    for j in range(i+1,cnt):
      str = itemset_1[i] + itemset_1[j]
      answer.add(str)
  return answer

In [ ]:
def generate_candidate(cur_node):
  for child in cur_node.child:
    if len(child.child) == 0:
      cnt = len(cur_node.child)
      for i in range(cnt):
        for j in range(i+1,cnt):
          new_node = Node(cur_node.child[i], cur_node.child[j].get_val())
          cur_node.child[i].insert_child(new_node)
      break
    else:
      generate_candidate(child)


In [ ]:
def remove_non_candidate(node, cur_level, level, bucket):
  if len(node.child) == 0:
    if cur_level < level:
      bucket.append(node)
      node.negate_candidate()
    return
  flag = True
  for child in node.child:
    remove_non_candidate(child, cur_level + 1, level, bucket)
    if child.is_candidate():
      flag = False
  
  if flag:
    bucket.append(node)
    node.negate_candidate()

In [ ]:
# def find_sup(itemset, idx, node):
#   for item in itemset[idx:len(itemset)]:
#     if item in node.child_dict:
#       dict_idx = node.child_dict[item]
#       find_sup(itemset, itemset.index(item) + 1, node.child[dict_idx - 1])

def find_sup(itemset, node):
  if len(node.child) == 0:
    node.inc_sup()
    return
  for child in node.child:
    val = child.get_val()
    if val in itemset and (itemset.index(val) + 1 < len(itemset)):
      find_sup(itemset[itemset.index(val) + 1:], child)



In [ ]:
f = open('Sample_Transactions','r')
itemset_1 = SortedSet()
itemset_2 = SortedSet()
mp = dict()
answer = dict()
transactions = []
bucket = []
sup_cnt = 3

In [ ]:
for transaction in f:
  transactions.append(transaction)
  itemset = re.split(',|\n',transaction)
  # print(2)
  # print(itemset)
  for item in itemset:
    if not item == '':
      if item in mp:
        mp[item] += 1
      else:
        mp[item] = 1

In [ ]:
for item in mp:
  if mp[item] >= sup_cnt:
    itemset_1.add(item)

answer[1] = itemset_1


In [ ]:
candidate_itemset_2 = generate_candidate_2(itemset_1)

6


In [ ]:
# print(candidate_itemset_2)
root = Node(None, None)
tree = trie(root, answer, candidate_itemset_2)

In [ ]:
# print(len(candidate_itemset_2))
# for transaction in transactions:
#   itemset = re.split(',|\n',transaction)
#   find_sup(itemset, root)

In [ ]:
item_len = 2

while True:
  itemset_2 = SortedSet()
  bucket = []
  if(len(root.child) == 0):
    break
  for transaction in transactions:
    itemset = re.split(',|\n',transaction)
    #Determine support count of candidates
    find_sup(itemset, root)

  print_path(root, "")
  # collect candidates having support count below threshold
  update_tree(root, "", sup_cnt, itemset_2, bucket)
  #Delete collected candidates
  for node in bucket:
    # print(node.get_val())
    tree.delete(node)
  print_path(root, "")

  print('itemlen ', item_len)
  #store answer
  answer[int(item_len)] = itemset_2

  item_len += 1

  print(answer)
  #generate candidates for next level
  generate_candidate(root)
  print_path(root, "")
  bucket.clear()

  #remove nodes that are below the current level
  remove_non_candidate(root, 0, item_len, bucket)
  for node in bucket:
    # print(node.get_val())
    tree.delete(node)
  print_path(root, "")

  # itemset_2.clear()
  # bucket.clear()

ab
ac
af
am
ap
bc
bf
bm
bp
cf
cm
cp
fm
fp
mp
ac
af
am
cf
cm
cp
fm
itemlen  2
{1: SortedSet(['a', 'b', 'c', 'f', 'm', 'p']), 2: SortedSet(['ac', 'af', 'am', 'cf', 'cm', 'cp', 'fm'])}
acf
acm
afm
am
cfm
cfp
cmp
cp
fm
acf
acm
afm
cfm
cfp
cmp
acf
acm
afm
cfm
cfp
cmp
acf
acm
afm
cfm
itemlen  3
{1: SortedSet(['a', 'b', 'c', 'f', 'm', 'p']), 2: SortedSet(['ac', 'af', 'am', 'cf', 'cm', 'cp', 'fm']), 3: SortedSet(['acf', 'acm', 'afm', 'cfm'])}
acfm
acm
afm
cfm
acfm
acfm
acfm
itemlen  4
{1: SortedSet(['a', 'b', 'c', 'f', 'm', 'p']), 2: SortedSet(['ac', 'af', 'am', 'cf', 'cm', 'cp', 'fm']), 3: SortedSet(['acf', 'acm', 'afm', 'cfm']), 4: SortedSet(['acfm'])}
acfm



In [ ]:
# print_path(root, "")
# update_tree(root, "", sup_cnt, itemset_2, bucket)
# for node in bucket:
#   # print(node.get_val())
#   tree.delete(node)
# print_path(root, "")
# answer[2] = itemset_2

In [ ]:
# print(answer)

In [ ]:
# generate_candidate(root)
# print_path(root, "")
# bucket.clear()


In [ ]:
# remove_non_candidate(root, 0, 3, bucket)
# for node in bucket:
#   # print(node.get_val())
#   tree.delete(node)
# print_path(root, "")